# **Criando Embeddings com Bertimbau**

In [3]:
#!pip install transformers
#!pip install sentence-transformers

from transformers import BertTokenizer, BertForMaskedLM
from nltk.tokenize import word_tokenize
from transformers import BertModel, BertTokenizer
import torch
from transformers import BertForSequenceClassification
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer 
from transformers import AutoModelForPreTraining  
from transformers import AutoModel 
from sentence_transformers import SentenceTransformer, util
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import pandas as pd
data = pd.read_csv('treino.txt', on_bad_lines='skip', header = None)

In [6]:
data = data.rename(columns={0: 'text'})


In [7]:
data.head()

,text
0,Uma criança risonha está segurando uma pistola...
1,Uma criança está segurando uma pistola de água
2,Os homens estão cuidadosamente colocando as ma...
3,Os homens estão colocando bagagens dentro do p...
4,Uma pessoa tem cabelo loiro e esvoaçante e est...


In [8]:
from nltk.tokenize import word_tokenize

text_tokenizer = data['text']
text_tokenizer = text_tokenizer.apply(word_tokenize)
text_tokenizer.head()

0    [Uma, criança, risonha, está, segurando, uma, ...
1    [Uma, criança, está, segurando, uma, pistola, ...
2    [Os, homens, estão, cuidadosamente, colocando,...
3    [Os, homens, estão, colocando, bagagens, dentr...
4    [Uma, pessoa, tem, cabelo, loiro, e, esvoaçant...
Name: text, dtype: object

In [9]:
model = SentenceTransformer('neuralmind/bert-base-portuguese-cased')
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=True)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [10]:
embeddings = text_tokenizer

In [11]:
embedding = model.encode(embeddings, convert_to_tensor=True)

In [12]:
class Embeddings:
  
  def __init__(self, sentence_1, sentence_2):
    self.sentence_1 = sentence_1
    self.sentence_2 = sentence_2
    self.tokenized_sent_1 = (self.sentence_1)
    self.tokenized_sent_2 = (self.sentence_2)
  
  def get_bertimbau_embedding(self):
    '''Creates word embeddings taken from BERT language representation
      returns: list, list
      Returns the BERT embeddings of the tokens of two sentences'''
    
    bertimbau_embedding_1  = embedding
    bertimbau_embedding_2  = embedding
    
    return bertimbau_embedding_1, bertimbau_embedding_2

In [13]:
class Embedding2Array:  
  
  def bertimbau(self,embedding): #Ricardo, atento a isto aqui
    ''' Creates the list of arrays of each corresponding word
    param
    embedding: list
    returns: list
      Returns the list of BERT embedding of each word 
    '''
#    word_array = []

#    for i in range(embedding[0].shape[1]):
#      word_array.append(embedding[0][0][i].tolist())
#    return word_array


    word_array = []
    for i in range(len(embedding)):
      word_array.append(embedding[i])
    return word_array

In [14]:
class SimilarityFunctions:

  def __init__(self, sentence_1, sentence_2, word_array_1, word_array_2):
    self.sentence_1 = sentence_1
    self.sentence_2 = sentence_2
    self.word_array_1 = word_array_1
    self.word_array_2 = word_array_2
  
  def cosine_similarity_matrix(self):
    '''
    Creates a matrix depicting the cosine distances between the words of two sentences
    returns: array
      Similarity matrix of words in two sentences
    '''
    matrix = np.zeros((len(self.word_array_1), len(self.word_array_2)))

    for i in range(0, len(self.word_array_1)):
      for j in range(0, len(self.word_array_2)):
        matrix[i][j]= scipy.spatial.distance.cosine(self.word_array_1[i], self.word_array_2[j])
    return matrix.T
  
  def plot_similarity_matrix(self, title):
    '''Plot the similarity matrix of two sentences
    param:
    title: str
      Labels the plot with the corresponding title
    returns: None
    '''
    x_labels, y_labels = word_tokenize(self.sentence_1), word_tokenize(self.sentence_2)
    similarity_matrix = self.cosine_similarity_matrix()
    sns.heatmap(similarity_matrix, vmin=0, vmax=1, xticklabels=x_labels, yticklabels=y_labels, cmap="YlGnBu", annot = True)
    plt.title(title)
    plt.show()    
  
  def get_similar_words(self):
    '''Prints similar word from second sentence for each word in the first sentence
    returns: list of similar words
    '''

    token_1 = word_tokenize(self.sentence_1)
    token_2 = word_tokenize(self.sentence_2)
    
    similarity_matrix = self.cosine_similarity_matrix()
    
    similar_word_dict = {}
    for row in range(0,len(similarity_matrix[0])):
      
      min_val = min(similarity_matrix.T[row]) #Here min value of transpose is found. To understand it print similarity matrix and find the logic
      index = (np.where(similarity_matrix.T[row] == min_val))[0]
      similar_word_list = []
      
      for i in range(0,len(index)):
        similar_word_list.append(token_2[index[i]])
        similar_word_dict[token_1[row]] = similar_word_list
      
    print('Similar words in two sentences are :', similar_word_dict)

In [15]:
class Main:
  def __init__(self, sentence_1, sentence_2):
    self.first_sentence = sentence_1
    self.second_sentence = sentence_2

  def bertimbau(self):
    '''Creates a list of BERT word embeddings of each sentence
    returns: list, list
      The list of embeddings of words of each sentence
    '''
    embed = Embeddings(self.first_sentence, self.second_sentence)
    embed2array = Embedding2Array()
    first_embedding, second_embedding = embed.get_bertimbau_embedding()
    first_array, second_array = embed2array.bertimbau(first_embedding), embed2array.bertimbau(second_embedding)
    return first_array, second_array

In [16]:
class AnalyzeWordEmbeddings:
  
  def __init__(self,sentence_1, sentence_2):
    self.sentence_1 = sentence_1
    self.sentence_2 = sentence_2
    
  def bertimbau(self):
    '''Plots similarity matrix using GPT embeddings and returns similar words in both the sentences
    returns: dict
      A dictionary matching the similar words from one sentence to other using GPT embeddings
    '''    
    main = Main(self.sentence_1, self.sentence_2)
    array_1, array_2 = main.bertimbau()
    similarityfunc = SimilarityFunctions(self.sentence_1, self.sentence_2, array_1, array_2)
    similarity_matrix = similarityfunc.cosine_similarity_matrix()
    similarityfunc.plot_similarity_matrix('Similarity plot using Bertimbau embeddings')
    similarityfunc.get_similar_words()

In [17]:
first_sentence_1 = "Os homens estão cuidadosamente colocando as malas no porta-malas de um carro"
first_sentence_2 = "Os homens estão colocando bagagens dentro do porta-malas de um carro" 

In [20]:
#analyze_word_embed_1 = AnalyzeWordEmbeddings(first_sentence_1, first_sentence_2)
#analyze_word_embed_1.bertimbau()

In [94]:
first_sentence_2 = "Uma criança risonha está segurando uma pistola de água e sendo espirrada com água"
first_sentence_2 = "Uma criança está segurando uma pistola de água" 

In [18]:
analyze_word_embed_1 = AnalyzeWordEmbeddings(first_sentence_2, first_sentence_2)
analyze_word_embed_1.bertimbau()